As discussed, Please see attached test and below question need you to complete it by next Thursday 17th August at midnight.  
 
The question is
 
In the attached data, you will find sample transactions that customers originated. 
The fields includes: date, ItemCategory, Price, Discount(if any), and Net price. 
Please, using the data, suggest the best item category for penetrating in should we decide to offer another item for sales. 
You can feel free to use any tools in your proposal, but please return the result, in written form, by next Thursday midnight.


Frequent Item Set Mining: Apriori, Eclat, FP-Growth

Algorithm

load file
    Load to program

Transform feature
    Parse date format to machine understanable form
        parse to timestamp
        separate date and time
    Handle missing row
        eg. remove or fill missing value by similarity of [Price, Discount, NMV]
        Just 8 missing row, maybe we decide to remove it
    Parse stuffCategories as one category in sequential
        eg. women_fashion,women_tops,women_fashion,women_tops => women_tops,women_tops
        Separate sequential of item into another column

Explore
    Plot to see findings
    holiday, not working time(), working time(8:00-18:00) and shopping time(18:00-22:00)

Predict

NaiveBayes
Apply Apriori Algorithm
    Write result to file
    Visualization
    Evaluate benchmark

In [3]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import util

# Transform data
parser = lambda date: pd.to_datetime(date)
df = dataset = pd.read_excel('dataset.xlsx')

In [4]:
# Parse date time
df['Date'] = df['Date'].apply(parser)
df['transaction_date'] = df['Date'].dt.date
df['is_weekend'] = df['Date'].dt.dayofweek.apply(util.is_weekend)
df['section_time'] = df['Date'].dt.hour.apply(util.quarter_time)

# Parse categories
df = data_without_null = df[pd.notnull(df['StuffCategories'])] #  clear null StuffCategories, cause it has only 8 rows
df['bought_categories'] = df['StuffCategories'].str.split(',')
df['bought_categories'] = df['bought_categories'].apply(util.unique_cat)

parent_categories = ['lifestyle', 'men_fashion', 'women_fashion']
bought_categories = [
    'all', 'lifestyle_crafts', 'lifestyle_for_home', 'lifestyle_gadget_tech', 
    'lifestyle_gift_ideas', 'lifestyle_other', 'lifestyle_phone_accessories', 'lifestyle_sports', 
    'lifestyle_stationeries', 'men_bags_wallets', 'men_glasses', 
    'men_hats', 'men_jewelry', 'men_pants', 'men_shoes', 'men_shorts', 'men_tops', 
    'men_underwear', 'men_watches', 'women_bags_wallets', 'women_dresses', 
    'women_glasses', 'women_hats', 'women_jackets_blazers', 
    'women_jewelry', 'women_other', 'women_pants_leggings', 'women_shoes', 
    'women_shorts', 'women_skirts', 'women_sports', 'women_swimwear', 'women_tops', 
    'women_watches', 'lifestyle', 'men_fashion', 'women_fashion'
]
df_cat = pd.DataFrame(index=None, columns=bought_categories)
df = pd.concat([df, df_cat]).fillna(0)
df = df.apply(util.word_matrix, axis=1)

# Drop all unused columns
df = df.drop('Date', axis=1)
df = df.drop('StuffCategories', axis=1)
df = df.drop('bought_categories', axis=1)
df


C:\Users\komunyong\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\komunyong\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Discount,NMV,Price,all,is_weekend,lifestyle,lifestyle_crafts,lifestyle_for_home,lifestyle_gadget_tech,lifestyle_gift_ideas,...,women_jewelry,women_other,women_pants_leggings,women_shoes,women_shorts,women_skirts,women_sports,women_swimwear,women_tops,women_watches
0,0.0,870.0,870.0,0,False,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,780.0,780.0,0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,600.0,600.0,0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,100.0,1300.0,1400.0,0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,460.0,460.0,0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0.0,1400.0,1400.0,0,False,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0.0,3600.0,3600.0,0,False,4,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0.0,880.0,880.0,0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0.0,1680.0,1680.0,0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0.0,4600.0,4600.0,1,False,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [ ]:
# Predict
df = df.drop(['transaction_date', 'transaction_time'], axis=1)

test_data_num = round(len(df) * 30 /100)
train_data_num = round(len(df) - test_data_num)
train_data = np.array(df[:test_data_num])
test_data = np.array(df[train_data_num:len(df)])

kmeans = KMeans(n_clusters=len(bought_categories)).fit(train_data)


kmeans.predict(test_data)
result = []
labels = kmeans.labels_
for label in labels:
    result = np.append(result, bought_categories[label])
result
df_result = pd.DataFrame(result)
df_predicted = data_without_null[train_data_num:len(data_without_null)]
dataset_suggested = pd.concat([df_predicted.reset_index(drop=True), df_result], axis=1)
dataset_suggested.to_csv('output.csv')
# data_without_null

# TODO:
# Write it to human understandable as csv file
# Separate train test as cross validation
# Discretization, [Discount, Net, Price]
# Solve imbalance data